In [2]:
import pandas as pd
import numpy as np
from pandas import DataFrame,Series

In [2]:
data =pd.read_csv("./movie_comments.csv",encoding='utf-8',dtype={'star': 'str','id':'str'})

In [3]:
data.dropna(axis=0, how='any', inplace=True)

In [4]:
data=data.loc[data['star']!='star']

In [5]:
data.head()

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1


In [6]:
status_dict = data['star'].unique().tolist()

In [10]:
status_dict=sorted(status_dict)

In [12]:
status_dict

['1', '2', '3', '4', '5']

In [14]:
data['transfromed']=data['star'].apply(lambda x : status_dict.index(x))

In [15]:
data.head()

,id,link,name,comment,star,transfromed
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1,0
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2,1
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2,1
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4,3
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1,0


In [17]:
def make_label(df):
    df["sentiment"] = df['star'].apply(lambda x: 1 if int(float(x))>2 else 0)

In [18]:
make_label(data)

In [19]:
X = data[['comment']]
y = data.sentiment

In [21]:
X

,comment
0,吴京意淫到了脑残的地步，看了恶心想吐
1,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...
3,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。
4,中二得很
...,...
261492,内容只能说一般。。女性角色怎么都这么悲催啊！！不过男猪脚很帅
261493,翘了三天班就窝在家里看小基片惹（手动拜拜.gif
261494,我喜欢女主角，希腊雕塑一般的面庞与身体。（在一部同志题材的电影中迷恋女主角好像很不应该吧）
261495,冲着颜值还可以看下去


In [20]:
y

0         0
1         0
2         0
3         1
4         0
         ..
261492    1
261493    1
261494    0
261495    1
261496    1
Name: sentiment, Length: 261494, dtype: int64

In [22]:
import jieba

In [23]:
def chinese_word_cut(txt):
    return " ".join(jieba.cut(txt))

In [24]:
X["cute_comment"] = X.comment.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\tb\AppData\Local\Temp\jieba.cache
Loading model cost 2.817 seconds.
Prefix dict has been built successfully.
D:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state=1)

In [61]:
X_train[:1]

,comment,cute_comment
217042,看了几分钟就知道是烂片，电影台正在播出，元素极其混搭得亮瞎双眼，还有歌舞，呵呵，其实冲着这个...,看 了 几分钟 就 知道 是 烂片 ， 电影台 正在 播出 ， 元素 极其 混搭得亮 瞎 双...


In [26]:
def get_custom_stopword(stop_word_file):
    with open(stop_word_file,encoding='utf-8') as f:
        stop_word = f.read()
    
    stop_word_list = stop_word.split('\n')
    custom_stopword = [i for i in stop_word_list]
    return custom_stopword

In [27]:
stop_words = []
with open('../baidu_stopwords.txt',encoding='utf-8') as f:
    for word in f.readlines():
        stop_words.append(word.strip())

In [28]:
stopwords = get_custom_stopword('../baidu_stopwords.txt')

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
max_df = 0.8 # 在超过这一比例的文档中出现的关键词（过于平凡），去除掉。
min_df = 100 # 在低于这一数量的文档中出现的关键词（过于独特），去除掉。

vect = CountVectorizer(max_df = max_df,
                       min_df = min_df,
                       token_pattern=u'(?u)\\b[^\\d\\W]\\w+\\b',
                       stop_words=frozenset(stopwords))

term_matrix = DataFrame(vect.fit_transform(X_train.cute_comment).toarray(), columns=vect.get_feature_names())


D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [35]:
term_matrix.shape

(196120, 2380)

In [36]:
from sklearn.naive_bayes import MultinomialNB
nb= MultinomialNB()

In [37]:
#vect = CountVectorizer()
#term_matrix = DataFrame(vect.fit_transform(X_train.cute_comment).toarray(),columns=vect.get_feature_names())

In [38]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(vect,nb)

In [39]:
pipe.steps

[('countvectorizer',
  CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                  dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                  lowercase=True, max_df=0.8, max_features=None, min_df=100,
                  ngram_range=(1, 1), preprocessor=None,
                  stop_words=frozenset({'', ' ', '--', '?', 'ZT', 'ZZ', "a's",
                                        'able', 'about', 'above', 'according',
                                        'accordingly', 'across', 'actually',
                                        'after', 'afterwards', 'again', 'against',
                                        "ain't", 'all', 'allow', 'allows',
                                        'almost', 'alone', 'along', 'already',
                                        'also', 'although', 'always', 'am', ...}),
                  strip_accents=None, token_pattern='(?u)\\b[^\\d\\W]\\w+\\b',
                  tokenizer=None, vocabulary=None)),
 ('m

In [3]:
from sklearn.model_selection import cross_val_score

In [45]:
cross_val_score(pipe,X_train.cute_comment,y_train,cv=5,scoring='accuracy').mean()

D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
D:\anaconda\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'di

0.8316897958481236

In [48]:
pipe.fit(X_train.cute_comment, y_train)

Pipeline(memory=None,
         steps=[('countvectorizer',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.8,
                                 max_features=None, min_df=100,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=frozenset({'', ' ', '--', '?', 'ZT',
                                                       'ZZ', "a's", 'able',
                                                       'about', 'above',
                                                       'according',
                                                       'accordingly', 'across',
                                                       'actually', 'after',
                                                       'afterwards', 'agai

In [49]:
y_pred=pipe.predict(X_test.cute_comment)

In [50]:
from sklearn import metrics

In [51]:
metrics.accuracy_score(y_test,y_pred)

0.8338177257013492

In [52]:
metrics.confusion_matrix(y_test,y_pred)

array([[ 5215,  7867],
       [ 2997, 49295]], dtype=int64)

In [4]:
from sklearn.externals import joblib

D:\anaconda\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [54]:
joblib.dump(pipe, 'model.normal')

['model.normal']

In [5]:
#load model
rfc2 = joblib.load('../model.normal')


D:\anaconda\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.22.2.post1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
D:\anaconda\lib\site-packages\sklearn\base.py:306: UserWarning: Trying to unpickle estimator Pipeline from version 0.22.2.post1 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [64]:
X_test[0:1]

,comment,cute_comment
90132,哪个草包写的剧本？黄晓明的台词大多不说人话 还以为他演的角色是耶稣呢 特效渣烂怂 舍不得花本...,哪个 草包 写 的 剧本 ？ 黄晓明 的 台词 大多 不说 人话 还 以为 他演 的 角...


In [6]:
import jieba

In [13]:
' '.join(jieba.cut('gfdsgf'))

'gfdsgf'

In [15]:
print(rfc2.predict([' '.join(list(jieba.cut('今天不高兴')))]))

[0]
